In [1]:
import pandas as pd
import numpy as np

In [2]:
def fill_age_with_meanstd(target, base):
    mean = base['Age'].mean()
    std = base['Age'].std()
    ammount = target['Age'].isna().sum()
    replace = np.random.randint(mean-std, mean+std, size =ammount)
    
    new_age = []
    it = iter(replace)
    for i in target['Age']:
        if np.isnan(i):
            new_age.append(next(it))
        else:
            new_age.append(i)
    target['Age'] = new_age

In [3]:
def fill_fare_with_meanstd(target, base):
    mean = base['Fare'].mean()
    std = base['Fare'].std()
    ammount = target['Fare'].isna().sum()
    replace = np.random.randint(mean-std, mean+std, size =ammount)
    
    new_age = []
    it = iter(replace)
    for i in target['Fare']:
        if np.isnan(i):
            new_age.append(next(it))
        else:
            new_age.append(i)
    target['Fare'] = new_age

In [4]:
def make_pclass_age_column(data):
    data['pclass_age'] = pd.cut(data['Age'], bins=[*range(0,81,5)],labels=[*range(0,16,1)])
    pclass_age = []
    for index, row in data.iterrows():  
        pclass_age.append(row['pclass_age'] * row['Pclass'])


    data['pclass_age'] = pclass_age
    data['pclass_age'] = data['pclass_age'].astype(int)

In [5]:
data = pd.read_csv('train.csv')

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [7]:
target = data['Survived']

In [8]:
data.drop(['PassengerId','Name','Parch','Ticket','Cabin','Embarked','Survived'], axis=1, inplace=True)

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Pclass  891 non-null    int64  
 1   Sex     891 non-null    object 
 2   Age     714 non-null    float64
 3   SibSp   891 non-null    int64  
 4   Fare    891 non-null    float64
dtypes: float64(2), int64(2), object(1)
memory usage: 34.9+ KB


In [10]:
fill_age_with_meanstd(data,data)

In [11]:
make_pclass_age_column(data)

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Pclass      891 non-null    int64  
 1   Sex         891 non-null    object 
 2   Age         891 non-null    float64
 3   SibSp       891 non-null    int64  
 4   Fare        891 non-null    float64
 5   pclass_age  891 non-null    int64  
dtypes: float64(2), int64(3), object(1)
memory usage: 41.9+ KB


In [13]:
from sklearn.preprocessing import StandardScaler

In [14]:
ss = StandardScaler()
ss.fit(data.loc[:,['Age', 'Fare','pclass_age']])
data.loc[:,['Age', 'Fare','pclass_age']] = ss.transform(data.loc[:,['Age', 'Fare','pclass_age']])

In [15]:
data['Sex'] = data['Sex'].apply(lambda x: 1 if x=='female' else 0)

In [16]:
data.head()

,Pclass,Sex,Age,SibSp,Fare,pclass_age
0,3,0,-0.546436,1,-0.502445,0.083757
1,1,1,0.638503,1,0.786845,-0.677747
2,3,1,-0.250201,0,-0.488854,0.540660
3,1,1,0.416327,1,0.420730,-0.830048
4,3,0,0.416327,0,-0.486337,0.997562


In [17]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

In [18]:
svc = SVC()

In [19]:
parameters_SVC = {'C':[1,10,100], 'kernel':['rbf', 'linear'], 'gamma':['scale','auto']}

In [20]:
gscv_SVC = GridSearchCV(svc, parameters_SVC,scoring='accuracy',verbose=2)

In [21]:
gscv_SVC.fit(data, target)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] END .......................C=1, gamma=scale, kernel=rbf; total time=   0.0s
[CV] END .......................C=1, gamma=scale, kernel=rbf; total time=   0.0s
[CV] END .......................C=1, gamma=scale, kernel=rbf; total time=   0.0s
[CV] END .......................C=1, gamma=scale, kernel=rbf; total time=   0.0s
[CV] END .......................C=1, gamma=scale, kernel=rbf; total time=   0.0s
[CV] END ....................C=1, gamma=scale, kernel=linear; total time=   0.0s
[CV] END ....................C=1, gamma=scale, kernel=linear; total time=   0.0s
[CV] END ....................C=1, gamma=scale, kernel=linear; total time=   0.0s
[CV] END ....................C=1, gamma=scale, kernel=linear; total time=   0.0s
[CV] END ....................C=1, gamma=scale, kernel=linear; total time=   0.0s
[CV] END ........................C=1, gamma=auto, kernel=rbf; total time=   0.0s
[CV] END ........................C=1, gamma=auto

GridSearchCV(estimator=SVC(),
             param_grid={'C': [1, 10, 100], 'gamma': ['scale', 'auto'],
                         'kernel': ['rbf', 'linear']},
             scoring='accuracy', verbose=2)

In [22]:
gscv_SVC.best_params_

{'C': 1, 'gamma': 'scale', 'kernel': 'rbf'}

In [23]:
gscv_SVC.best_score_

0.8159186491745652

In [24]:
forest = RandomForestClassifier()

In [25]:
parameters_forest = {'n_estimators': [10,100,1000],'criterion': ['gini', 'entropy'],'max_depth':[None, 5, 10]}

In [26]:
gscv_forest = GridSearchCV(forest, parameters_forest,scoring='accuracy',verbose=2)

In [27]:
gscv_forest.fit(data,target)

Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV] END ....criterion=gini, max_depth=None, n_estimators=10; total time=   0.0s
[CV] END ....criterion=gini, max_depth=None, n_estimators=10; total time=   0.0s
[CV] END ....criterion=gini, max_depth=None, n_estimators=10; total time=   0.0s
[CV] END ....criterion=gini, max_depth=None, n_estimators=10; total time=   0.0s
[CV] END ....criterion=gini, max_depth=None, n_estimators=10; total time=   0.0s
[CV] END ...criterion=gini, max_depth=None, n_estimators=100; total time=   0.2s
[CV] END ...criterion=gini, max_depth=None, n_estimators=100; total time=   0.2s
[CV] END ...criterion=gini, max_depth=None, n_estimators=100; total time=   0.2s
[CV] END ...criterion=gini, max_depth=None, n_estimators=100; total time=   0.2s
[CV] END ...criterion=gini, max_depth=None, n_estimators=100; total time=   0.2s
[CV] END ..criterion=gini, max_depth=None, n_estimators=1000; total time=   1.9s
[CV] END ..criterion=gini, max_depth=None, n_est

GridSearchCV(estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [None, 5, 10],
                         'n_estimators': [10, 100, 1000]},
             scoring='accuracy', verbose=2)

In [28]:
gscv_forest.best_params_

{'criterion': 'gini', 'max_depth': 10, 'n_estimators': 10}

In [29]:
gscv_forest.best_score_

0.8204444165463561

Therefore, the best predictor for this test was the Random Forest with max_depth of 10, with 100 trees using gini criterion.

# Training final estimator

In [30]:
final_estimator = RandomForestClassifier(10,max_depth=10)

In [31]:
final_estimator.fit(data,target)

RandomForestClassifier(max_depth=10, n_estimators=10)

# Preprocessing pipeline

In [32]:
def preprocessing_titanic(target_data, base_data):
    target_data.set_index("PassengerId", inplace=True)
    target_data = target_data.loc[:,['Pclass','Sex','Age','SibSp','Fare']]
    
    if len(target_data[target_data['Age'].isna()]['Age']) > 0:
        fill_age_with_meanstd(target_data, base_data)
        
    if len(target_data[target_data['Fare'].isna()]['Fare']) > 0:
        fill_fare_with_meanstd(target_data,base_data)
    
    make_pclass_age_column(target_data)
    
    ss = StandardScaler()
    ss.fit(base_data.loc[:,['Age', 'Fare','pclass_age']])
    target_data.loc[:,['Age', 'Fare','pclass_age']] = ss.transform(target_data.loc[:,['Age', 'Fare','pclass_age']])

    target_data['Sex'] = target_data['Sex'].apply(lambda x: 1 if x=='female' else 0)
    return target_data

In [33]:
base_data = pd.read_csv('train.csv')
fill_age_with_meanstd(base_data, base_data)
make_pclass_age_column(base_data)

In [34]:
test_data = pd.read_csv('test.csv')

In [35]:
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [36]:
test_data = preprocessing_titanic(test_data, base_data)

In [37]:
test_data.head()

,Pclass,Sex,Age,SibSp,Fare,pclass_age
PassengerId,,,,,,
892,3,0,0.362062,0,-0.490783,0.973452
893,3,1,1.288568,1,-0.507479,2.336490
894,2,0,2.400376,0,-0.453367,1.882144
895,3,0,-0.193842,0,-0.474005,0.519107
896,3,1,-0.564445,1,-0.401017,0.064761


In [38]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 892 to 1309
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Pclass      418 non-null    int64  
 1   Sex         418 non-null    int64  
 2   Age         418 non-null    float64
 3   SibSp       418 non-null    int64  
 4   Fare        418 non-null    float64
 5   pclass_age  418 non-null    float64
dtypes: float64(3), int64(3)
memory usage: 22.9 KB


In [41]:
pred = final_estimator.predict(test_data)

In [42]:
answers = pd.read_csv('gender_submission.csv')

In [43]:
answers.set_index('PassengerId',inplace=True)

In [44]:
answers

,Survived
PassengerId,
892,0
893,1
894,0
895,0
896,1
...,...
1305,0
1306,1
1307,0


In [45]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [46]:
confusion_matrix(answers, pred)

array([[246,  20],
       [ 34, 118]])

In [47]:
accuracy_score(answers, pred)

0.8708133971291866

In [52]:
results = pd.DataFrame(pred,index=answers.index, columns=['Survived'])

In [53]:
results.to_csv('my_results.csv')